In [ ]:
from bs4 import BeautifulSoup
import pandas as pd
import requests

In [ ]:
print("Hello world")

In [ ]:
url = "http://ufcstats.com/event-details/7956f026e2672c47"

In [ ]:
r = requests.get(url)

In [ ]:
html_content = r.content


In [ ]:
soup = BeautifulSoup(r.content, 'html.parser')


In [ ]:
soup

In [ ]:
location = None
info_box = soup.find("div", class_="b-list__info-box")
if info_box:
    
    items = info_box.find_all("li", class_="b-list__box-list-item")
    for item in items:
     title = item.find("i", class_="b-list__box-item-title")
    if title and "Location" in title.text:
        location = item.get_text(strip=True).replace("Location:", "").strip()
location

In [ ]:
date = None
if info_box:
    items = info_box.find_all("li", class_="b-list__box-list-item")
    for item in items:
        title = item.find("i", class_="b-list__box-item-title")
        if title and "Date" in title.text:
            date = item.get_text(strip=True).replace("Date:", "").strip()
            break

print(date)

In [ ]:
r = requests.get("http://ufcstats.com/event-details/8db1b36dde268ef6")
soup = BeautifulSoup(r.content, 'html.parser')

fight_links = []

rows = soup.find_all("tr", class_="b-fight-details__table-row")
for row in rows:
    link = row.get("data-link")
    if link:
        fight_links.append(link)

print(fight_links)

In [ ]:
ref_r = requests.get(fight_links[2])
soup_ref = BeautifulSoup(ref_r.content, 'html.parser')
soup_ref.find_all('span')[0].get_text(strip="True")

In [ ]:
table = soup.find_all("table",class_='b-fight-details__table')[0]

In [ ]:
table

In [ ]:
headers = table.find_all("th")

In [ ]:
headers

In [ ]:
headers = [i.text  for i in headers ]
headers

In [ ]:
headers = [i.strip(' ').strip('\n').strip(' ')  for i in headers ]
headers

In [ ]:
df = pd.DataFrame(columns=headers)
df

In [ ]:
data = soup.find_all('td')
data

In [ ]:
link = "http://ufcstats.com/event-details/92c96df8bdab5fea"
r = requests.get(link)
soup = BeautifulSoup(r.content, 'html.parser')
soup

In [ ]:
import re
import pandas as pd

# scrape ufc fight stats from one event link

In [ ]:
def ufc_scraper(link):

    r = requests.get(link)
    soup = BeautifulSoup(r.content, 'html.parser')

    # Extract location once per event
    location = None
    info_box = soup.find("div", class_="b-list__info-box")
    if info_box:
        items = info_box.find_all("li", class_="b-list__box-list-item")
        for item in items:
            title = item.find("i", class_="b-list__box-item-title")
            if title and "Location" in title.text:
                location = item.get_text(strip=True).replace("Location:", "").strip()
    date = None
    if info_box:
        items = info_box.find_all("li", class_="b-list__box-list-item")
        for item in items:
            title = item.find("i", class_="b-list__box-item-title")
            if title and "Date" in title.text:
                date = item.get_text(strip=True).replace("Date:", "").strip()
                break



    rows = soup.select("tbody tr")
    all_fights = []

    def clean_text(txt):
        return re.sub(r"\s+", " ", txt).strip()

    for row in rows:
        cols = row.find_all("td")
        if len(cols) < 10:
            continue

        fighters = [p.text.strip() for p in cols[1].select("p")]
        kd = [clean_text(p.text) for p in cols[2].select("p")]
        sig_str = [clean_text(p.text) for p in cols[3].select("p")]
        td = [clean_text(p.text) for p in cols[4].select("p")]
        sub = [clean_text(p.text) for p in cols[5].select("p")]
       

        link = row.get("data-link")
        ref_name = None
        if link:
            ref_r = requests.get(link)
            soup_ref = BeautifulSoup(ref_r.content, 'html.parser')
            ref_name=soup_ref.find_all('span')[0].get_text(strip="True")
            
        
        res = None
        winner = None

        if clean_text(cols[7].text) == 'CNC':
            res = 'nc'
            winner = "-"

        weight_class_cell = cols[6]

        # Detect title fight based on belt icon
        img = weight_class_cell.select_one("img")
        title_fight = False
        if img and "belt.png" in img.get("src", ""):
            title_fight = True
            
            

        fight = {
            "fighter_1": fighters[0],
            "fighter_2": fighters[1],
            "date": date,
            "location": location,
            "referee": ref_name,
            "title_fight":title_fight,
            "fighter_1_kd": int(kd[0]) if kd[0].isdigit() else 0,
            "fighter_2_kd": int(kd[1]) if kd[1].isdigit() else 0,
            "fighter_1_sig_str": int(sig_str[0]),
            "fighter_2_sig_str": int(sig_str[1]),
            "fighter_1_td": int(td[0]),
            "fighter_2_td": int(td[1]),
            "fighter_1_sub": int(sub[0]),
            "fighter_2_sub": int(sub[1]),
            "result": res or cols[0].text.strip(),
            "winner": winner or (fighters[0] if cols[0].text.strip().lower() == "win" else fighters[1]),
            "weight_class": clean_text(cols[6].text),
            "method": clean_text(cols[7].text),
            "rounds": int(cols[8].text.strip()),
            "time": cols[9].text.strip(),
            
        }

        all_fights.append(fight)

    # Convert list of dicts to DataFrame
    df = pd.DataFrame(all_fights)
    return df

    

# Scrape the title fight indicators

In [ ]:
def ufc_scraper_title(link):

    r = requests.get(link)
    soup = BeautifulSoup(r.content, 'html.parser')

   



    rows = soup.select("tbody tr")
    all_fights = []

    def clean_text(txt):
        return re.sub(r"\s+", " ", txt).strip()

    for row in rows:
        cols = row.find_all("td")
        if len(cols) < 10:
            continue

       


        


        weight_class_cell = cols[6]

        # Detect title fight based on belt icon
        img = weight_class_cell.select_one("img")
        title_fight = False
        if img and "belt.png" in img.get("src", ""):
            title_fight = True
            
            

        fight = {

            "title_fight":title_fight

        }

        all_fights.append(fight)

    # Convert list of dicts to DataFrame
    df = pd.DataFrame(all_fights)
    return df


def full_Scraper_title(last_page):

    df_original = pd.DataFrame()

    for i in range(1 ,int(last_page)+1):
        
        r = requests.get("http://ufcstats.com/statistics/events/completed?page="+str(i))
        soup = BeautifulSoup(r.content, 'html.parser')

        events=soup.find_all("a", class_="b-link")
        event_links = []
        for a in events:  # `events` is your list of <a> tags
            link = a.get("href")
            if link:
                event_links.append(link)

        if i == 1:
            event_links = event_links[1:]

        for url in event_links:
            
            df1 = ufc_scraper_title(url)
            df_original = pd.concat([df_original,df1], ignore_index=True)

    return df_original

    

In [ ]:
df_original = pd.DataFrame()

In [ ]:
title_column = full_Scraper_title(29)

In [ ]:
title_column

In [ ]:
ufc_scraper_title("http://ufcstats.com/event-details/27541033b97c076d")

In [ ]:
ufc_scraper("http://ufcstats.com/event-details/27541033b97c076d")

# Complete ufc fight records scraper

In [ ]:

def full_Scraper(last_page):

    df_original = pd.DataFrame()

    for i in range(1 ,int(last_page)+1):
        
        r = requests.get("http://ufcstats.com/statistics/events/completed?page="+str(i))
        soup = BeautifulSoup(r.content, 'html.parser')

        events=soup.find_all("a", class_="b-link")
        event_links = []
        for a in events:  # `events` is your list of <a> tags
            link = a.get("href")
            if link:
                event_links.append(link)

        if i == 1:
            event_links = event_links[1:]

        for url in event_links:
            
            df1 = ufc_scraper(url)
            df_original = pd.concat([df_original,df1], ignore_index=True)

    return df_original
            
            
                
        
    

In [ ]:
df_original = pd.DataFrame() 

In [ ]:
or_df_frame=full_Scraper(29)

In [ ]:
e=1

In [ ]:
r = requests.get("http://ufcstats.com/statistics/events/completed?page=2")
soup = BeautifulSoup(r.content, 'html.parser')

events=soup.find_all("a", class_="b-link")
event_links = []
for a in events:  # `events` is your list of <a> tags
    link = a.get("href")
    if link:
        event_links.append(link)

if e == 1:
    event_links = event_links[1:]

for url in event_links:
            
    df1 = ufc_scraper(url)
    df_original = pd.concat([df_original,df1], ignore_index=True)

In [ ]:
df_original

In [ ]:
or_df_frame

In [ ]:
or_df_frame.to_csv("./dataset/ufc_data.csv",index=False)

# Scraping fighter details

In [ ]:
def get_fighter_links(url):
    r = requests.get(url)
    soup = BeautifulSoup(r.content, "html.parser")

    table = soup.find("table", class_="b-statistics__table")
    rows = table.find("tbody").find_all("tr", class_="b-statistics__table-row")

    fighter_links = []

    for row in rows:
        # link is in the first <a> tag inside the row
        a = row.find("a")
        if a and a.get("href"):
            fighter_links.append(a["href"])

    return fighter_links


# Example usage:
url = "http://ufcstats.com/statistics/fighters?char=a"
links = get_fighter_links(url)
print(links)



In [ ]:
def scrape_fighter_basic_and_stats(url):
    res = requests.get(url)
    soup = BeautifulSoup(res.content, "html.parser")

    # ---------- NAME ----------
    name = soup.select_one(".b-content__title-highlight")
    name = name.text.strip() if name else None

    # ---------- RECORD ----------
    record = soup.select_one(".b-content__title-record")
    record = record.text.replace("Record:", "").strip() if record else None

    # ---------- PHYSICAL INFO ----------
    info_list = soup.select(".b-list__info-box li")

    def extract(label):
        for li in info_list:
            text = li.text.strip()
            if label in text:
                return text.replace(label, "").strip()
        return None

    height = extract("Height:")
    weight = extract("Weight:")
    reach = extract("Reach:")
    stance = extract("STANCE:")
    dob = extract("DOB:")

    # ---------- CAREER STATS ----------
    career_labels = {
        "SLpM": "SLpM:",
        "StrAcc": "Str. Acc.:",
        "SApM": "SApM:",
        "StrDef": "Str. Def.:",
        "TDAvg": "TD Avg.:",
        "TDAcc": "TD Acc.:",
        "TDDef": "TD Def.:",
        "SubAvg": "Sub. Avg.:",
    }

    career_stats = {}

    for key, label in career_labels.items():
        # find <i> that contains the label
        stat_icon = soup.find("i", string=lambda s: s and label in s)
        if stat_icon:
            value = stat_icon.find_next_sibling(text=True)
            career_stats[key] = value.strip() if value else None
        else:
            career_stats[key] = None

    # ---------- FINAL OUTPUT ----------
    return {
        "name": name,
        "record": record,
        "height": height,
        "weight": weight,
        "reach": reach,
        "stance": stance,
        "dob": dob,
        **career_stats
    }



In [ ]:
fighter_url="http://ufcstats.com/fighter-details/1338e2c7480bdf9e"
scrape_fighter_basic_and_stats(fighter_url)

In [ ]:
import string

# Lowercase alphabets
lowercase_alphabets = list(string.ascii_lowercase)
print(lowercase_alphabets)

In [ ]:

def full_fighter_detail_scraper():

    fighter_details = []

    for i in lowercase_alphabets:

        page_url='http://ufcstats.com/statistics/fighters?char='+str(i)+'&page=all'
    
        fighter_url_set = get_fighter_links(page_url)
    
        
    
        for u in fighter_url_set:
    
            fighter = scrape_fighter_basic_and_stats(u)
            fighter_details.append(fighter)

    df = pd.DataFrame(fighter_details)
    return df
        

    
    

In [ ]:
df_fighters_final = pd.DataFrame()

In [ ]:
df_fighters_final = full_fighter_detail_scraper()

In [ ]:
f1_df = full_fighter_detail_scraper('z',1)
df_fighters_final = pd.concat([df_fighters_final,f1_df], ignore_index=True)

In [ ]:
df_fighters_final

In [ ]:
df_fighters_final[df_fighters_final['name']=='Umar Nurmagomedov']

In [ ]:
df_fighters_final.info()

In [ ]:
df_fighters_final.to_csv('./dataset/ufc_fighters.csv', index=False) 

In [ ]:
df_fighters_final.drop(columns='StrDef', inplace=True)

In [ ]:
df_fights = pd.read_csv("./dataset/ufc_data.csv")

In [ ]:
df_fights.head()

In [ ]:
df_fighters_final

In [ ]:
df = df_fights.copy()

df = df.merge(df_fighters_final, left_on='fighter_1', right_on='name', how='left', suffixes=("", "_1"))
df = df.merge(df_fighters_final, left_on='fighter_2', right_on='name', how='left', suffixes=("", "_2"))


In [ ]:
df.head()

In [ ]:
df_fighters_final[df_fighters_final['name']== 'Belal Muhammad']

In [ ]:
df.to_csv('ufc_full.csv', index=False) 

In [ ]:
df1 = pd.read_csv("./dataset/ufc_full.csv")

In [ ]:
df1.head()

In [ ]:
link = "http://ufcstats.com/event-details/8db1b36dde268ef6"
r = requests.get(link)
soup = BeautifulSoup(r.content, 'html.parser')
soup

In [ ]:
events_df = pd.read_csv("./dataset/ufc_data.csv")
title_df = title_column.copy()

In [ ]:
events_df.shape,title_column.shape

In [ ]:
title_column.head(10)

In [ ]:
events_df['title_fight'] = title_df

In [ ]:
events_df

In [ ]:
events_df.to_csv("./dataset/ufc1.csv", index=False)

In [ ]:
def scrape_str_def(url):
    res = requests.get(url)
    soup = BeautifulSoup(res.content, "html.parser")

    # find <i> tag with "Str. Def"
    label = soup.find("i", string=lambda t: t and "Str. Def" in t)

    if not label:
        return None

    # extract value which appears immediately after the <i> tag
    value = label.find_next_sibling(text=True)

    if value:
        return {"StrDef": value.strip()}   # returns "54%"
    return None

In [ ]:
lowercase_alphabets = ['a']
print(lowercase_alphabets)

In [ ]:

def full_strDef_scraper(i):

    fighter_details = []

    

    page_url='http://ufcstats.com/statistics/fighters?char='+str(i)+'&page=all'
    
    fighter_url_set = get_fighter_links(page_url)
    
        
    
    for u in fighter_url_set:
    
        strDef = scrape_str_def(u)
        fighter_details.append(strDef)

    df = pd.DataFrame(fighter_details)
    return df
        

    
    

In [ ]:
df_t = pd.DataFrame()

In [ ]:
df_strDef = full_strDef_scraper('z')
df_t = pd.concat([df_t,df_strDef], ignore_index=True)

In [ ]:
df_t

In [ ]:
df_fighters = pd.read_csv("./dataset/ufc_fighters.csv")

In [ ]:
df_fighters.shape

In [ ]:
df_fighters['StrDef'] = df_t

In [ ]:
df_fighters.head()

In [ ]:
df_fighters.tail()

In [ ]:
df_fighters.to_csv("./dataset/ufc_fighters1.csv", index=False)